In [ ]:
!pip install llama-index-vector-stores-chroma chromadb

In [ ]:
!pip install llama-index-llms-openrouter

In [3]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.node_parser import SentenceSplitter


In [4]:

# === 1. Set up free embedding model (Hugging Face) ===
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# === 2. Set up free LLM via OpenRouter ===
Settings.llm = OpenRouter(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",  # Free tier model
    api_key="",
    max_tokens=512
)

In [6]:

# === 3. Load documents ===
documents = SimpleDirectoryReader("data").load_data()


In [7]:
# Optional: Better chunking
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=50)
nodes = node_parser.get_nodes_from_documents(documents)


In [8]:

# === 4. Create index with Chroma (local vector DB) ===
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex(nodes=nodes, vector_store=vector_store)

In [ ]:
# === 5. Query Engine ===
query_engine = index.as_query_engine(similarity_top_k=3)

In [10]:







# === 6. Ask questions ===
response = query_engine.query("who is shahab khan?")
print("\nAnswer:")
print(response)

response = query_engine.query("How does OpenRouter help?")
print("\nAnswer:")
print(response)


Answer:
Shahab Khan is a Full-Stack AI Engineer and Computer Science student with expertise in machine learning, deep learning, generative AI, and AI automation. He has experience developing end-to-end AI solutions using frameworks like TensorFlow, Hugging Face, LangChain, and n8n. His skills include parameter-efficient fine-tuning of models like LLaMA 3.2, building chatbot workflows, and integrating LLMs and APIs for automation. He is also pursuing a Bachelor's degree in Computer Science and has upcoming certifications in Azure AI Fundamentals, Deep Learning, and Machine Learning with Python.

Answer:
I can't answer that question.
